In [3]:
import os
import json
from dotenv import load_dotenv
from tqdm import tqdm
import google.generativeai as genai

# Load environment variables and configure Gemini
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("gemini-1.5-pro")

def get_selling_recommendations(stocks, cash_needed):
    prompt = f"""You are a stock portfolio optimization expert. I need to generate ${cash_needed:,.2f} in cash by selling some underperforming stocks.

    Here are my stocks that are predicted to decline:

    """

    # Add each stock's details to the prompt--have to add cuz of the for loop, but could just combine and put under 1 bigger for loop if needed
    for stock in tqdm(stocks, desc="Processing Stocks"):
        prompt += f"""
        Stock: {stock['symbol']}
        Current Price: ${stock['current_price']:.2f}
        Shares Owned: {stock['quantity']}
        Current Total Value: ${stock['current_price'] * stock['quantity']:,.2f}
        Predicted Changes:
        - Tomorrow: {stock['next_day_change']:.1f}%
        - Next Week: {stock['next_week_change']:.1f}%
        - Next Month: {stock['next_month_change']:.1f}%
        """

    prompt += f"""
        Please analyze these stocks and tell me:
        1. Which stocks I should sell
        2. How many shares of each to sell
        3. Why you made these recommendations

        Provide your response in this JSON format:
        {{
            "recommendations": [
                {{
                    "symbol": "STOCK_SYMBOL",
                    "shares_to_sell": number,
                    "expected_cash": number,
                    "reasoning": "brief explanation"
                }}
            ],
            "total_cash_generated": number,
            "strategy_explanation": "brief overall explanation"
        }}

        Make sure the total_cash_generated meets or slightly exceeds ${cash_needed:,.2f}.

        Prioritize selling stocks with the worst long-term outlook.

        """

    response = model.generate_content(prompt)

    return json.loads(response.text)


In [4]:
if __name__ == "__main__":
    underperforming_stocks = [
        {
            "symbol": "AAPL",
            "current_price": 150.0,
            "quantity": 100,
            "next_day_change": -1.5,
            "next_week_change": -3.2,
            "next_month_change": -6.8
        },
        {
            "symbol": "GOOGL",
            "current_price": 2800.0,
            "quantity": 10,
            "next_day_change": -0.8,
            "next_week_change": -2.1,
            "next_month_change": -7.2
        },
        {
            "symbol": "META",
            "current_price": 300.0,
            "quantity": 50,
            "next_day_change": -2.1,
            "next_week_change": -4.5,
            "next_month_change": -8.5
        }
    ]

    # Get recommendations for generating $10,000
    recommendations = get_selling_recommendations(underperforming_stocks, 10000)

    # Print results
    print("\nSelling Recommendations:")
    print(json.dumps(recommendations, indent=2))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)